In [30]:
import os
import json
import gradio as gr
import base64
from PIL import Image
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
from dotenv import load_dotenv
from openai import OpenAI
import random

In [31]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print("All good🤙")
else:
    print('check your key')
    
openai = OpenAI()
model = 'gpt-4o-mini'

All good🤙


In [32]:
# Enhanced system prompt for language tutoring
system_prompt = '''You are BroLingo, a cool and friendly language tutor specializing in English, Chinese (Mandarin), German, Spanish, and French.

Your teaching style:
- Keep it chill and friendly - you're like a language buddy, not a strict teacher
- Use casual language and be encouraging
- Make mistakes feel normal and part of the learning process
- Celebrate even small wins
- Use fun examples and real-world contexts
- Be patient but keep things engaging

When helping students:
- Ask about their level in a friendly way
- Give practical, everyday examples
- Make pronunciation tips easy to remember
- Suggest fun practice activities
- Share cool cultural facts
- Keep the vibe positive and motivating

Available tools help you with:
- Vocabulary definitions and usage
- Grammar explanations
- Translations between languages
- Conversation practice topics
- Language level assessment

Always be encouraging and make learning fun - you're the bro who makes languages feel easy!'''


In [33]:
# Language data and vocabulary
language_voices = {
    'english': 'alloy',
    'spanish': 'nova',
    'french': 'shimmer',
    'german': 'echo',
    'chinese': 'fable'
}

vocabulary_database = {
    'english': {
        'beginner': ['hello', 'goodbye', 'please', 'thank you', 'water', 'food', 'house', 'family'],
        'intermediate': ['appreciate', 'understand', 'important', 'different', 'experience', 'information'],
        'advanced': ['sophisticated', 'comprehensive', 'inevitable', 'substantial', 'preliminary']
    },
    'spanish': {
        'beginner': ['hola', 'adiós', 'por favor', 'gracias', 'agua', 'comida', 'casa', 'familia'],
        'intermediate': ['apreciar', 'entender', 'importante', 'diferente', 'experiencia', 'información'],
        'advanced': ['sofisticado', 'comprensivo', 'inevitable', 'sustancial', 'preliminar']
    },
    'french': {
        'beginner': ['bonjour', 'au revoir', 's\'il vous plaît', 'merci', 'eau', 'nourriture', 'maison', 'famille'],
        'intermediate': ['apprécier', 'comprendre', 'important', 'différent', 'expérience', 'information'],
        'advanced': ['sophistiqué', 'compréhensif', 'inévitable', 'substantiel', 'préliminaire']
    },
    'german': {
        'beginner': ['hallo', 'auf wiedersehen', 'bitte', 'danke', 'wasser', 'essen', 'haus', 'familie'],
        'intermediate': ['schätzen', 'verstehen', 'wichtig', 'verschieden', 'erfahrung', 'information'],
        'advanced': ['sophisticated', 'umfassend', 'unvermeidlich', 'wesentlich', 'vorläufig']
    },
    'chinese': {
        'beginner': ['你好', '再见', '请', '谢谢', '水', '食物', '房子', '家庭'],
        'intermediate': ['欣赏', '理解', '重要', '不同', '经验', '信息'],
        'advanced': ['复杂', '全面', '不可避免', '重要', '初步']
    }
}

In [34]:
grammar_rules = {
    'english': {
        'present_simple': 'Use for habits, facts, and general truths. Form: Subject + base verb (+s for 3rd person)',
        'past_simple': 'Use for completed actions in the past. Form: Subject + past form of verb',
        'articles': 'Use "a/an" for indefinite, "the" for definite. "A" before consonants, "an" before vowels'
    },
    'spanish': {
        'ser_estar': 'SER for permanent states, ESTAR for temporary states and locations',
        'gender': 'Nouns have gender (masculine/feminine). Adjectives must agree in gender and number',
        'articles': 'el/la (definite), un/una (indefinite). Must agree with noun gender'
    },
    'french': {
        'gender': 'All nouns have gender (masculine/feminine). Adjectives must agree',
        'articles': 'le/la/les (definite), un/une/des (indefinite). Must agree with noun',
        'pronunciation': 'Many final consonants are silent. Liaison connects words in speech'
    },
    'german': {
        'cases': 'Four cases: Nominativ, Akkusativ, Dativ, Genitiv. Articles change based on case',
        'word_order': 'Verb is second element in main clauses. Complex verb positions in subordinate clauses',
        'capitalization': 'All nouns are capitalized, not just proper nouns'
    },
    'chinese': {
        'tones': 'Mandarin has 4 tones plus neutral tone. Tone changes meaning completely',
        'word_order': 'Generally Subject-Verb-Object, but can be flexible',
        'particles': 'Particles like 了, 的, 吗 are crucial for meaning and grammar'
    }
}

conversation_topics = {
    'beginner': ['introducing yourself', 'ordering food', 'asking for directions', 'shopping', 'weather'],
    'intermediate': ['hobbies and interests', 'travel experiences', 'work and career', 'cultural differences'],
    'advanced': ['current events', 'philosophy and ideas', 'professional discussions', 'complex emotions']
}

In [35]:
# Tool functions for language learning
def get_vocabulary_definition(word, language='english', level='beginner'):
    """Get definition and example for a vocabulary word"""
    print(f"Getting definition for '{word}' in {language} (level: {level})")
    
    # This is a simplified version - in a real app, you'd use a proper dictionary API
    definition_prompt = f"Provide a clear, {level}-level definition of the word '{word}' in {language}, include pronunciation guide and an example sentence."
    
    return {
        'word': word,
        'language': language,
        'level': level,
        'definition': f"Definition for '{word}' in {language} at {level} level"
    }

def translate_text(text, source_lang, target_lang):
    """Translate text between supported languages"""
    print(f"Translating '{text}' from {source_lang} to {target_lang}")
    
    return {
        'original': text,
        'source_language': source_lang,
        'target_language': target_lang,
        'translation': f"Translation of '{text}' from {source_lang} to {target_lang}"
    }

def explain_grammar(grammar_topic, language):
    """Explain grammar rules for specific language"""
    print(f"Explaining {grammar_topic} in {language}")
    
    rules = grammar_rules.get(language, {})
    explanation = rules.get(grammar_topic, f"Grammar explanation for {grammar_topic} in {language}")
    
    return {
        'topic': grammar_topic,
        'language': language,
        'explanation': explanation
    }

def get_conversation_starter(level='beginner', language='english'):
    """Get conversation practice topics"""
    print(f"Getting conversation starter for {level} level in {language}")
    
    topics = conversation_topics.get(level, conversation_topics['beginner'])
    topic = random.choice(topics)
    
    return {
        'level': level,
        'language': language,
        'topic': topic,
        'starter': f"Let's practice {topic} in {language}"
    }

def generate_vocabulary_quiz(language='english', level='beginner', num_words=5):
    """Generate a vocabulary quiz"""
    print(f"Generating {num_words} word quiz in {language} ({level} level)")
    
    words = vocabulary_database.get(language, {}).get(level, [])
    quiz_words = random.sample(words, min(num_words, len(words)))
    
    return {
        'language': language,
        'level': level,
        'words': quiz_words,
        'quiz_type': 'vocabulary_practice'
    }

In [36]:
# Tool definitions for OpenAI
vocabulary_function = {
    "name": "get_vocabulary_definition",
    "description": "Get definition, pronunciation, and example usage of a word in the specified language and level",
    "parameters": {
        "type": "object",
        "properties": {
            "word": {"type": "string", "description": "The word to define"},
            "language": {"type": "string", "description": "Language (english, spanish, french, german, chinese)"},
            "level": {"type": "string", "description": "Proficiency level (beginner, intermediate, advanced)"}
        },
        "required": ["word"],
        "additionalProperties": False
    }
}

translation_function = {
    "name": "translate_text",
    "description": "Translate text between any of the supported languages",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {"type": "string", "description": "Text to translate"},
            "source_lang": {"type": "string", "description": "Source language"},
            "target_lang": {"type": "string", "description": "Target language"}
        },
        "required": ["text", "source_lang", "target_lang"],
        "additionalProperties": False
    }
}

grammar_function = {
    "name": "explain_grammar",
    "description": "Explain grammar rules and concepts for a specific language",
    "parameters": {
        "type": "object",
        "properties": {
            "grammar_topic": {"type": "string", "description": "Grammar topic to explain"},
            "language": {"type": "string", "description": "Language for grammar explanation"}
        },
        "required": ["grammar_topic", "language"],
        "additionalProperties": False
    }
}

conversation_function = {
    "name": "get_conversation_starter",
    "description": "Get conversation practice topics and starters for language learning",
    "parameters": {
        "type": "object",
        "properties": {
            "level": {"type": "string", "description": "Proficiency level (beginner, intermediate, advanced)"},
            "language": {"type": "string", "description": "Target language for conversation practice"}
        },
        "required": ["level", "language"],
        "additionalProperties": False
    }
}

quiz_function = {
    "name": "generate_vocabulary_quiz",
    "description": "Generate vocabulary quiz for practice",
    "parameters": {
        "type": "object",
        "properties": {
            "language": {"type": "string", "description": "Language for the quiz"},
            "level": {"type": "string", "description": "Difficulty level"},
            "num_words": {"type": "integer", "description": "Number of words in quiz"}
        },
        "required": ["language", "level"],
        "additionalProperties": False
    }
}

In [37]:
tools = [
    {'type': 'function', 'function': vocabulary_function},
    {'type': 'function', 'function': translation_function},
    {'type': 'function', 'function': grammar_function},
    {'type': 'function', 'function': conversation_function},
    {'type': 'function', 'function': quiz_function}
]

def handle_tool_call(message):
    """Handle tool calls from OpenAI"""
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        if function_name == 'get_vocabulary_definition':
            result = get_vocabulary_definition(
                arguments.get('word'),
                arguments.get('language', 'english'),
                arguments.get('level', 'beginner')
            )
            return {
                "role": "tool",
                "content": f"Word: {result['word']}\nLanguage: {result['language']}\nLevel: {result['level']}\nDefinition: {result['definition']}",
                "tool_call_id": tool_call.id
            }
        
        elif function_name == 'translate_text':
            result = translate_text(
                arguments.get('text'),
                arguments.get('source_lang'),
                arguments.get('target_lang')
            )
            return {
                "role": "tool",
                "content": f"Original ({result['source_language']}): {result['original']}\nTranslation ({result['target_language']}): {result['translation']}",
                "tool_call_id": tool_call.id
            }
        
        elif function_name == 'explain_grammar':
            result = explain_grammar(
                arguments.get('grammar_topic'),
                arguments.get('language')
            )
            return {
                "role": "tool",
                "content": f"Grammar Topic: {result['topic']} in {result['language']}\nExplanation: {result['explanation']}",
                "tool_call_id": tool_call.id
            }
        
        elif function_name == 'get_conversation_starter':
            result = get_conversation_starter(
                arguments.get('level', 'beginner'),
                arguments.get('language', 'english')
            )
            return {
                "role": "tool",
                "content": f"Conversation Practice ({result['level']} {result['language']})\nTopic: {result['topic']}\nStarter: {result['starter']}",
                "tool_call_id": tool_call.id
            }
        
        elif function_name == 'generate_vocabulary_quiz':
            result = generate_vocabulary_quiz(
                arguments.get('language', 'english'),
                arguments.get('level', 'beginner'),
                arguments.get('num_words', 5)
            )
            return {
                "role": "tool",
                "content": f"Vocabulary Quiz ({result['language']} - {result['level']})\nWords to practice: {', '.join(result['words'])}",
                "tool_call_id": tool_call.id
            }
    
    return None

In [38]:
def talker(message, language='english'):
    """Text-to-speech with language-specific voices"""
    # Set a custom directory for temporary files
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir
    
    # Create the folder if it doesn't exist
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)
    
    # Choose voice based on language
    voice = language_voices.get(language.lower(), 'alloy')
    
    response = openai.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [39]:
def chat(history, target_language='english'):
    """Main chat function with language learning focus"""
    # Ensure history is a list
    if not history:
        history = []
        
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        tool_response = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=model, messages=messages)
    
    reply = response.choices[0].message.content
    history.append({"role": "assistant", "content": reply})
    
    # Use language-specific voice for TTS
    talker(reply, target_language)
    
    return history

In [40]:
def transcribe_audio(audio_file):
    """Convert audio to text using OpenAI Whisper"""
    if audio_file is None:
        return ""
    
    try:
        with open(audio_file, "rb") as audio:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio,
                response_format="text"
            )
        return transcript.strip()
    except Exception as e:
        print(f"Transcription error: {e}")
        return ""

In [41]:
def process_audio_input(audio_file, history, target_language):
    """Process audio input: transcribe, then chat"""
    if audio_file is None:
        return None, history
    
    # Transcribe the audio
    transcribed_text = transcribe_audio(audio_file)
    
    if not transcribed_text:
        return None, history
    
    # Add user message to history
    history = history or []
    history.append({"role": "user", "content": transcribed_text})
    
    # Process through chat system
    updated_history = chat(history, target_language)
    
    # Return empty audio (to clear input) and updated history
    return None, updated_history

In [42]:
def process_text_input(message, history, target_language):
    """Handle text input and process through chat"""
    if not message.strip():
        return "", history
    
    # Initialize history if None
    history = history or []
    
    # Add to history
    history.append({"role": "user", "content": message})
    
    # Process through chat
    updated_history = chat(history, target_language)
    
    return "", updated_history

In [53]:
# Enhanced Gradio Interface
with gr.Blocks(
    title="BroLingo - Your Language Learning Buddy",
    theme=gr.themes.Soft(),
    css="""
    .gradio-container {
        max-width: 1200px !important;
        margin: 0 auto;
    }
    .chat-container {
        border-radius: 15px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .input-section {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 20px;
        border-radius: 15px;
        margin-top: 10px;
    }
    .voice-section {
        background: rgba(255, 255, 255, 0.1);
        padding: 15px;
        border-radius: 10px;
        margin-top: 10px;
        border: 1px solid rgba(255, 255, 255, 0.2);
    }
    """
) as ui:
    
    # Header Section
    with gr.Row():
        gr.Markdown(
            """
            # 🤙 BroLingo - Your Chill Language Learning Buddy
            *Mastering English, Spanish, French, German, and Chinese - no stress, just progress!*
            """,
            elem_classes="header"
        )
    
    # Language Selection
    with gr.Row():
        target_language = gr.Dropdown(
            choices=['english', 'spanish', 'french', 'german', 'chinese'],
            value='english',
            label="🎯 Target Language for Learning",
            info="Choose the language you want to practice",
            scale=2
        )
        with gr.Column(scale=1):
            gr.Markdown("### 🚀 Ready to learn?")
    
    # Chat Interface
    with gr.Row():
        chatbot = gr.Chatbot(
            height=500,
            type="messages",
            label="💬 Chat with BroLingo",
            show_label=True,
            elem_classes="chat-container",
            show_copy_button=True
        )
    
    # Input Section with better layout
    with gr.Group(elem_classes="input-section"):
        gr.Markdown("### 💬 Your Turn")
        
        # Text Input Row
        with gr.Row():
            entry = gr.Textbox(
                show_label=False,
                placeholder="Type your message here... 📝",
                lines=2,
                max_lines=4,
                scale=8
            )
            send_btn = gr.Button(
                "Send 📤",
                variant="primary",
                scale=1,
                size="lg"
            )
        
        # Voice Input Section (below text input)
        with gr.Group(elem_classes="voice-section"):
            gr.Markdown("#### 🎤 Or speak your message:")
            with gr.Row():
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    show_label=False,
                    scale=3
                )
                with gr.Column(scale=1):
                    gr.Markdown("*Click the mic to record your voice!*")
    
    # Action Buttons
    with gr.Row():
        clear = gr.Button("🗑️ Clear Chat", variant="secondary", scale=1)
        quick_help = gr.Button("❓ Quick Help", variant="primary", scale=1)
        practice_mode = gr.Button("🎯 Practice Mode", variant="stop", scale=1)
    
    # Status indicator
    with gr.Row():
        status = gr.Textbox(
            value="Ready to chat! 🤙",
            label="Status",
            interactive=False,
            scale=1
        )
    
    # Event handlers
    def process_text_with_status(text, history, language):
        if not text.strip():
            return "", history, "Please enter a message! 📝"
        
        # Your existing process_text_input function logic here
        result_text, result_history = process_text_input(text, history, language)
        return result_text, result_history, "Message sent! 🚀"
    
    def process_audio_with_status(audio, history, language):
        if audio is None:
            return None, history, "No audio recorded 🎤"
        
        # Your existing process_audio_input function logic here
        result_audio, result_history = process_audio_input(audio, history, language)
        return result_audio, result_history, "Audio processed! 🎵"
    
    # Text input events
    entry.submit(
        process_text_with_status,
        inputs=[entry, chatbot, target_language],
        outputs=[entry, chatbot, status]
    )
    
    send_btn.click(
        process_text_with_status,
        inputs=[entry, chatbot, target_language],
        outputs=[entry, chatbot, status]
    )
    
    # Audio input events
    audio_input.change(
        process_audio_with_status,
        inputs=[audio_input, chatbot, target_language],
        outputs=[audio_input, chatbot, status]
    )
    
    # Clear chat
    def clear_chat():
        return None, "Chat cleared! 🧹"
    
    clear.click(
        clear_chat,
        outputs=[chatbot, status]
    )
    
    # Help function with better formatting
    def show_help():
        help_message = """
        **Yo! Welcome to BroLingo!** 🤙
        
        I'm here to help you learn languages without the stress! Here's what we can do together:
        
        **📚 Vocab Stuff**: Ask me what words mean, get examples, or let's do some vocab quizzes  
        **🔤 Grammar Help**: I'll break down grammar rules so they actually make sense  
        **🔄 Translation**: Need something translated? I got you covered  
        **💬 Conversation**: Let's chat and practice real conversations  
        **🎯 Fun Quizzes**: Test your skills with some friendly quizzes  
        
        **Try asking me things like:**
        - "Bro, what does 'hola' mean?"
        - "How do I use French present tense?"
        - "Translate 'What's up?' to German"
        - "Let's practice ordering pizza in Spanish"
        - "Hit me with a beginner vocab quiz in Chinese"
        
        **Pro tip**: Use the mic to practice your pronunciation - I'll help you sound like a native! 🎤
        
        Ready to level up your language game? Let's go! 🚀
        """
        return [{"role": "assistant", "content": help_message}], "Help loaded! 📖"
    
    quick_help.click(
        show_help,
        outputs=[chatbot, status]
    )
    
    # Practice mode function
    def start_practice():
        practice_message = """
        **🎯 Practice Mode Activated!**
        
        Let's get you practicing! Here are some quick practice options:
        
        **1. Pronunciation Practice** 🗣️
        - Use the microphone to practice saying words
        - I'll help you with pronunciation tips
        
        **2. Quick Vocab Quiz** 🧠
        - Test your vocabulary knowledge
        - Get instant feedback
        
        **3. Conversation Starters** 💬
        - Practice real-world conversations
        - Build confidence speaking
        
        What would you like to practice today?
        """
        return [{"role": "assistant", "content": practice_message}], "Practice mode ready! 🎯"
    
    practice_mode.click(
        start_practice,
        outputs=[chatbot, status]
    )

# Launch with better configuration
ui.launch(
    inbrowser=True,
    share=False,
    show_error=True,
    quiet=False
)

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.
